In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
data_file = './/Amazon_Unlocked_Mobile.csv'
n = 413000  
s = 3000
skip = sorted(random.sample(range(1,n),n-s))


df = pd.read_csv( data_file, delimiter = ",", skiprows = skip)
#41만개의 데이터 중 3000개의 데이터만 뽑아서 분석을 진행 

In [3]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,5,my autistic son is in his glory lol thank you ...,0.0
1,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,2,The SIM card from the provider would not work ...,2.0
2,"4 Inch Touch Screen Cell Phone Unlocked, Andro...",NaN,23.90,4,My sim card slide all the way up in the slot. ...,NaN
3,"4 Inch Touch Screen Cell Phone Unlocked, Andro...",NaN,23.90,5,Gave the phone as a birthday gift. My friend s...,2.0
4,"5.5"" Cellphone Dual Sim Unlocked Android 5.1 Q...",JUNING,99.99,1,Didnt offer clear and concise instructions on ...,0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [4]:
df.isnull().sum()

Product Name      0
Brand Name      545
Price            41
Rating            0
Reviews           0
Review Votes     93
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [7]:
df
#결측치들을 제거하니 41만개의 데이터에서 33만개의 데이터로 축소되었음
#아마 하나의 row에 결측치가 한개라도 존재하면 해당 row를 제거하는 방법을 수행했을것으로 예상 

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,5,my autistic son is in his glory lol thank you ...,0.0
1,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,2,The SIM card from the provider would not work ...,2.0
4,"5.5"" Cellphone Dual Sim Unlocked Android 5.1 Q...",JUNING,99.99,1,Didnt offer clear and concise instructions on ...,0.0
5,"5.5"" Phone Unlocked Dual Sim Quad Core-JUNING ...",JUNING,99.99,5,"5.5"" Phone Unlocked Dual Sim Quad Core-JUNING...",0.0
6,"5.5"" Unlocked GSM Cell Phones Android 5.1 MTK6...",JUNING,99.99,1,"I bought this phone "" 5.5""unlocked GSM phone"" ...",0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


### Rating Mapping

In [8]:
Review_mapping =  {0: '0',1 : '0', 2 :'0',3 : '0', 4 : '1', 5 : '1'}
Review_mapping
#0~3점까지는 0으로 4~5점까지는 1로 맵핑 

{0: '0', 1: '0', 2: '0', 3: '0', 4: '1', 5: '1'}

In [9]:
Rating = lambda x: Review_mapping.get(x,x)
df['Rating']=df.Rating.map(Rating)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df['Rating'].unique()

array(['1', '0'], dtype=object)

In [11]:
df
#rating이 잘변환된것을 확인할 수 있다.

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,1,my autistic son is in his glory lol thank you ...,0.0
1,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,0,The SIM card from the provider would not work ...,2.0
4,"5.5"" Cellphone Dual Sim Unlocked Android 5.1 Q...",JUNING,99.99,0,Didnt offer clear and concise instructions on ...,0.0
5,"5.5"" Phone Unlocked Dual Sim Quad Core-JUNING ...",JUNING,99.99,1,"5.5"" Phone Unlocked Dual Sim Quad Core-JUNING...",0.0
6,"5.5"" Unlocked GSM Cell Phones Android 5.1 MTK6...",JUNING,99.99,0,"I bought this phone "" 5.5""unlocked GSM phone"" ...",0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,I returned it because it did not meet my needs...,0.0


In [12]:
df.to_csv("Sentiment Analysis Final .csv")

### 텍스트 데이터 분리

In [13]:
df.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [14]:
a = df.iloc[:,3]
print(a)
# a는 평점
a.astype('int64')
#맵핑을 한 rating이 object로 되어있어 int로 변환을 하였음 

0       1
1       0
4       0
5       1
6       0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3192, dtype: object


0       1
1       0
4       0
5       1
6       0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3192, dtype: int64

In [15]:
b = df.iloc[:,4]
# b는 리뷰데이터(텍스트)

In [16]:
doc = pd.concat([a,b], axis = 1)
#기존의 데이터에서 rating과 review로 구성된 새로운 데이터셋을 구성 

### Text Cleaning

In [17]:
def review_to_wordlist(review, remove_stopwords = True):

    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)


In [18]:
clean_reviews = []
for review in doc['Reviews']:
    clean_reviews.append( " ".join(review_to_wordlist(review)))

In [19]:
len(clean_reviews)
#각 row마다 텍스트 전처리가 잘된것을 확인 

3192

In [20]:
clean_reviews = pd.DataFrame(clean_reviews)

In [21]:
clean_reviews.columns = ['Reviews']
clean_reviews
#컬럼명을 Reviews로 설정 

,Reviews
0,autist son glori lol thank
1,sim card provid would work phone return phone
2,didnt offer clear concis instruct put sim card...
3,phone unlock dual sim quad core june android c...
4,bought phone unlock gsm phone ship africa phon...
...,...
3187,anoth great deal great price
3188,ok
3189,pass everi drop test onto porcelain tile
3190,return meet need seem good select other


In [22]:
df = pd.concat([clean_reviews, doc['Rating']], axis = 1)
df.isnull().sum()
#무작정 concat을 사용했기때문에 결측값이 발생 
#index를 맞춰주지않아 문제가 발생한 것으로 보인다.

Reviews    631
Rating     631
dtype: int64

In [23]:
clean_reviews.reset_index(drop=True , inplace=True)
clean_reviews
#인덱스를 초기화해줌 

,Reviews
0,autist son glori lol thank
1,sim card provid would work phone return phone
2,didnt offer clear concis instruct put sim card...
3,phone unlock dual sim quad core june android c...
4,bought phone unlock gsm phone ship africa phon...
...,...
3187,anoth great deal great price
3188,ok
3189,pass everi drop test onto porcelain tile
3190,return meet need seem good select other


In [24]:
b = doc['Rating']

In [25]:
b.reset_index(drop = True, inplace = True)
b
#rating의 인덱스를 초기화해줌 

0       1
1       0
2       0
3       1
4       0
       ..
3187    1
3188    0
3189    1
3190    0
3191    1
Name: Rating, Length: 3192, dtype: object

In [26]:
Data = pd.concat([clean_reviews, doc['Rating']], axis = 1)
Data.isnull().sum()
#인덱스를 초기화하여 각각의 데이터프레임을 결합을 하였으며 결측값이 발생하지 않음 
#인덱스를 초기화해주지않아서 문제가 발생을 하였다

Reviews    0
Rating     0
dtype: int64

In [27]:
Data

,Reviews,Rating
0,autist son glori lol thank,1
1,sim card provid would work phone return phone,0
2,didnt offer clear concis instruct put sim card...,0
3,phone unlock dual sim quad core june android c...,1
4,bought phone unlock gsm phone ship africa phon...,0
...,...,...
3187,anoth great deal great price,1
3188,ok,0
3189,pass everi drop test onto porcelain tile,1
3190,return meet need seem good select other,0


### 인덱스와 원핫인코딩
1. 각 단어를 토큰화해서 해당 토큰들을 인덱스화 
2. 인덱스한 단어들을 원핫인코딩으로 변환 

In [28]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [64]:
a=list(Data['Reviews'])

In [67]:
c = ' '.join(a)
print(c)

autist son glori lol thank sim card provid would work phone return phone didnt offer clear concis instruct put sim card got bad eyesight put microsim sim slot tri remov tweezer phone wont power phone unlock dual sim quad core june android cellphon blue arriv good condit good packag excit final receiv cell phone gift nephew back home receiv immedi began charg abl start phone success oper system work perfect necessari updat make fast smooth oper camera power produc excel pictur built microphon good record audio clear video record playback function seamless smooth also like size phone good solid feel weight cell phone came screen protector alreadi instal addit screen protector packag cell phone easi use especi like dual sim option work phone number separ person number phone afford look great love cell phone would absolut recommend anyon receiv cell phone reduc discount price exchang honest unbias review bought phone unlock gsm phone ship africa phone never work waist money love phone phon

In [68]:
words = word_tokenize(c)
print(words)
#전처리한 것들을 단어단위로 자름 

['autist', 'son', 'glori', 'lol', 'thank', 'sim', 'card', 'provid', 'would', 'work', 'phone', 'return', 'phone', 'didnt', 'offer', 'clear', 'concis', 'instruct', 'put', 'sim', 'card', 'got', 'bad', 'eyesight', 'put', 'microsim', 'sim', 'slot', 'tri', 'remov', 'tweezer', 'phone', 'wont', 'power', 'phone', 'unlock', 'dual', 'sim', 'quad', 'core', 'june', 'android', 'cellphon', 'blue', 'arriv', 'good', 'condit', 'good', 'packag', 'excit', 'final', 'receiv', 'cell', 'phone', 'gift', 'nephew', 'back', 'home', 'receiv', 'immedi', 'began', 'charg', 'abl', 'start', 'phone', 'success', 'oper', 'system', 'work', 'perfect', 'necessari', 'updat', 'make', 'fast', 'smooth', 'oper', 'camera', 'power', 'produc', 'excel', 'pictur', 'built', 'microphon', 'good', 'record', 'audio', 'clear', 'video', 'record', 'playback', 'function', 'seamless', 'smooth', 'also', 'like', 'size', 'phone', 'good', 'solid', 'feel', 'weight', 'cell', 'phone', 'came', 'screen', 'protector', 'alreadi', 'instal', 'addit', 'scree

In [70]:
from collections import Counter
WordInfo = Counter(words)
print(WordInfo)
#데이터가 핸드폰에 대한 데이터임으로 Phone이 가장 많은 빈도를 보임 

Counter({'phone': 4056, 'work': 991, 'use': 894, 'good': 828, 'great': 742, 'like': 576, 'one': 567, 'get': 466, 'screen': 453, 'love': 402, 'batteri': 391, 'would': 372, 'time': 356, 'new': 341, 'price': 307, 'buy': 297, 'need': 295, 'problem': 295, 'product': 283, 'nice': 280, 'card': 279, 'app': 276, 'realli': 276, 'sim': 268, 'call': 268, 'excel': 258, 'want': 255, 'bought': 253, 'camera': 252, 'look': 249, 'well': 248, 'charg': 243, 'got': 240, 'unlock': 238, 'day': 234, 'back': 233, 'purchas': 228, 'go': 218, 'even': 217, 'also': 213, 'iphon': 209, 'perfect': 207, 'month': 205, 'much': 197, 'make': 195, 'better': 193, 'receiv': 192, 'qualiti': 186, 'first': 180, 'android': 174, 'recommend': 173, 'case': 173, 'fast': 172, 'take': 166, 'expect': 166, 'featur': 165, 'still': 165, 'littl': 164, 'return': 163, 'life': 163, 'text': 163, 'could': 162, 'came': 160, 'servic': 157, 'thing': 156, 'order': 155, 'samsung': 153, 'come': 152, 'easi': 151, 'set': 151, 'replac': 151, 'tri': 149, 

In [73]:
max_words = 2000
maxlen = 30

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts()

TypeError: fit_on_texts() missing 1 required positional argument: 'texts'

In [57]:
b = []
for i in range(len(a)):
    b.append(a[i].split(','))
    
print(b)
#각 줄마다 리스트로 리스트로 변환하였음 

[['autist son glori lol thank'], ['sim card provid would work phone return phone'], ['didnt offer clear concis instruct put sim card got bad eyesight put microsim sim slot tri remov tweezer phone wont power'], ['phone unlock dual sim quad core june android cellphon blue arriv good condit good packag excit final receiv cell phone gift nephew back home receiv immedi began charg abl start phone success oper system work perfect necessari updat make fast smooth oper camera power produc excel pictur built microphon good record audio clear video record playback function seamless smooth also like size phone good solid feel weight cell phone came screen protector alreadi instal addit screen protector packag cell phone easi use especi like dual sim option work phone number separ person number phone afford look great love cell phone would absolut recommend anyon receiv cell phone reduc discount price exchang honest unbias review'], ['bought phone unlock gsm phone ship africa phone never work wais

In [ ]:
max_words = 10000
maxlen = 200

In [30]:
print(type(Data['Reviews']))
#이거 [[]]이런 형태로 나와야 NLP에서 강의를 들었던것처럼 됨 

<class 'pandas.core.series.Series'>


In [39]:
text = sent_tokenize(Data['Reviews'])
print(text)

TypeError: expected string or bytes-like object

In [29]:
train_docs, test_docs = train_test_split(Data, test_size = 0.3)
#학습과 테스트를 위해 원본 데이터를 7:3비율로 분할 

In [218]:
rf_X_train = vect_train_data.toarray()
rf_y_train = (np.array(train_docs['Rating']))
rf_X_test = vect_test_data.toarray()
rf_y_test = (np.array(test_docs['Rating']))

### Random Forest

In [219]:
from sklearn.ensemble import RandomForestClassifier

In [220]:
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2020)
forest 
#데이터가 너무 커서 최적의 파라미터들을 찾기보다는 default로 두는 것이 가장 좋은 선택이라는 판단 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=2020, verbose=0,
                       warm_start=False)

In [221]:
forest = forest.fit(rf_X_train, rf_y_train)

In [222]:
result = forest.predict(rf_X_test)

In [223]:
rf_y_test = rf_y_test.astype('int64')

In [225]:
result = result.astype('int64')

In [226]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    #Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [227]:
get_clf_eval(rf_y_test, result)
#랜덤포레스트모델에서는 재현율은 아주 좋은 성능을 보여주며 f1 score는 재현율에 영향을 받기때문에 높은 값으로 나온것롤 보인다 
#하지만 다른 성능지표에서는 그렇게 좋지 못한 성능을 보여주고 있다.

오차행렬:
 [[ 56 242]
 [ 54 615]]

정확도: 0.6939
정밀도: 0.7176
재현율: 0.9193
F1: 0.8060
AUC: 0.5536


### XGBOOST

In [326]:
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [327]:
dtrain = xgb.DMatrix(data = rf_X_train, label = rf_y_train)
dtest = xgb.DMatrix(data= rf_X_test, label = rf_y_test)

#numpy형태의 학습 데이터셋과 테스트 데이터셋을 DMatirx로 변환
#xgboost 모델을 학습시키기위해서는 Dmatrix로 변환을 해야하는것 같음 

In [328]:
params = {'max_depth' : 4,
          'eta' : 0.1,
          'objective' : 'binary:logistic',
          'eval_metric' : 'logloss',
          'early_stoppings' : 30}

num_rounds = 100
#eta : 학습률 
#손실함수 평가지표 : logloss
#파라미터를 

In [329]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params = params, 
                      dtrain = dtrain, 
                      num_boost_round = num_rounds,
                      evals = wlist)

[13:04:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66371	eval-logloss:0.68640
[1]	train-logloss:0.63909	eval-logloss:0.68041
[2]	train-logloss:0.61831	eval-logloss:0.66941
[3]	train-logloss:0.60042	eval-logloss:0.66270
[4]	train-logloss:0.58357	eval-logloss:0.66270
[5]	train-logloss:0.56980	eval-logloss:0.65563
[6]	train-logloss:0.55825	eval-logloss:0.65019
[7]	train-logloss:0.54613	eval-logloss:0.65111
[8]	train-logloss:0.53590	eval-logloss:0.64448
[9]	train-logloss:0.52593	eval-logloss:0.64576
[10]	train-logloss:0.51775	eval-logloss:0.64241
[11]	train-logloss:0.51028	eval-logloss:0.64236
[12]	train-logloss:0.50375	eval-logloss:0.63845
[13]	train-logloss:0.

In [362]:
pred_probs = xgb_model.predict(dtest)
#예측값을 확률로 출력 

In [356]:
preds = [1 if x > 0.4 else 0 for x in pred_probs]

In [357]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
#Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [358]:
get_clf_eval(rf_y_test, preds)
#재현율에서는 아주 높은 값을 보여주고 있지만 나머지값들에 대해서는 좋지 못한 결과를 보여준다
#또한 AUC가 0.5정도면 좋지 못한 모델이라는 것을 보여준다

오차행렬:
 [[  9 289]
 [ 10 659]]

정확도: 0.6908
정밀도: 0.6951
재현율: 0.9851
F1: 0.8151
AUC: 0.5076


### LSTM

In [236]:
X_train = rf_X_train
y_train = rf_y_train 
X_test = rf_X_test
y_test = rf_y_test

In [269]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [320]:
embed_size = 64
max_features = 1500


model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#6개의 layer로 구성되어 있으며 성능을 좋게 내기 위해 학습 모델 코드를 참조하였음 

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, None, 64)          96000     
_________________________________________________________________
bidirectional_16 (Bidirectio (None, None, 64)          24832     
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 64)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 20)                1300      
_________________________________________________________________
dropout_16 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 21        
Total params: 122,153
Trainable params: 122,153
Non-trainable params: 0
_______________________________________________

In [359]:
batch_size = 256
epochs = 10
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)
#Validation set을 train셋에서 30%를 두고 학습을 진행 

Train on 1578 samples, validate on 677 samples
Epoch 1/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6119 - accuracy: 0.7003 - val_loss: 0.6196 - val_accuracy: 0.6898
Epoch 2/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6129 - accuracy: 0.7003 - val_loss: 0.6194 - val_accuracy: 0.6898
Epoch 3/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6117 - accuracy: 0.7003 - val_loss: 0.6195 - val_accuracy: 0.6898
Epoch 4/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6110 - accuracy: 0.7003 - val_loss: 0.6198 - val_accuracy: 0.6898
Epoch 5/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6106 - accuracy: 0.7003 - val_loss: 0.6200 - val_accuracy: 0.6898
Epoch 6/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6120 - accuracy: 0.7003 - val_loss: 0.6193 - val_accuracy: 0.6898
Epoch 7/10
1578/1578 [==============================] - 3s 2ms/step - loss: 0.6100 - accu

In [360]:
score, acc =model.evaluate(X_test, y_test, batch_size = batch_size)

967/967 [==============================] - 1s 623us/step


In [361]:
print('Test Score:' , score)
print('Test Accuracy:' , acc)
#69.18%의 정확도를 보여주고 있으며 좋지못한 성능을 보여준다 
#추후 성능을 올릴방법을 찾아서 성능을 올릴예정

Test Score: 0.6171910514382813
Test Accuracy: 0.6918303966522217
